In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
data = pd.read_csv('facies_vectors_0.csv')
feature_names = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
facies_names = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS', 'WS', 'D', 'PS', 'BS']
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00', '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']
data = data.fillna(data['PE'].mean())

In [13]:
import numpy as np
import xgboost as xgb
import math
from sklearn.preprocessing import OneHotEncoder

print('start running example to used customized objective function')

params = {'max_depth': 2, 'eta': 0.1, 'silent': 1,
          'objective': 'multi:softprob', 'num_class': 9, 'learning_rate':0.1}

num_round = 2
def my_softmax(preds, dtrain):
    labels = dtrain.get_label()
    labels = OneHotEncoder(sparse=False, n_values=9).fit_transform(labels.reshape(-1, 1))
    grad = preds - labels
    hess = preds * (1.0-preds)
    dtrain_len = dtrain.num_row()
    
#     L = [[0]*9]*dtrain.num_row()
#     H = [[0]*9]*dtrain.num_row()
#     in_ = 0
#     for object_ in range(1, dtrain_len-1):
#         ind1 = np.where(preds[object_-1] == max(preds[object_-1]))[0][0]
#         max1 = max(preds[object_])
#         ind = np.where(preds[object_] == max1)[0][0]
#         max2 = 0
#         for index in range(len(preds[object_])):
#             if (preds[object_][index] > max2):
#                 ind2 = np.where(preds[object_] == preds[object_][index])[0][0]
#                 if (ind2 != ind):
#                     max2 = preds[object_][index]
# #         ind2 = np.where(preds[object_+1] == max(preds[object_+1]))[0][0]
# #         if(ind1 == ind2):
#         if (abs(max2 - max1) < 0.02):
#             in_ += 1
#             for index in range(9):
#                 for class_ in range(9):
#                     if (class_ != ind1):
#                         L[object_][index] = L[object_][index] - preds[object_][class_] * preds[object_][index]
#                         H[object_][index] = H[object_][index] + 2 * preds[object_][index] * preds[object_][index] * preds[object_][class_] - preds[object_][index] * preds[object_][class_] 
#                 H[object_][index] = H[object_][index] - 2 * preds[object_][index] * preds[object_][index] + preds[object_][index]
#                 L[object_][index] = L[object_][index] + preds[object_-1][index]
#     print("grad ", grad[0])
#     print("L ", L[0])
#     grad = grad - 0.001 *np.array(L)
#     print(in_, " ", dtrain_len-2)
#     hess = hess - 0.001*np.array(H)
    return grad.flatten(), hess.flatten()

start running example to used customized objective function


In [14]:
def number_of_outlier(y_res):
    outliers = 0
    if y_res[0] != y_res[1]:
        outliers += 1
    if y_res[-1] != y_res[-2]:
        outliers += 1
    for index in range(1,len(y_res)-1):
        if ((y_res[index] != y_res[index-1]) and (y_res[index] != y_res[index+1])):
            outliers += 1
    return outliers/len(y_res)
            

In [15]:
def most_similar_classes(y_res, y_test):
    for class_1 in range(9):
        for class_2 in range(class_1 + 1,9):
            for index in range(len(y_test)):
                if (((y_test[index] == class_1) and (y_res[index] == class_2)) or ((y_test[index] == class_2) and (y_res[index] == class_1))):
                    similar_classes[class_1, class_2] += 1                

In [16]:
def recount_most_similar_classes():
    for class_1 in range(9):
        for class_2 in range(class_1 + 1,9):
            similar_classes[class_1, class_2] = similar_classes[class_1, class_2] * 2 /(sum_classes[class_1] + sum_classes[class_2])

In [17]:
def find_sum_classes(y_res, y_test):
    for class_ in range(9):
        for index in range(len(y_test)):
            if ((y_test[index] == class_) or (y_res[index] == class_)):
                sum_classes[class_] += 1

In [19]:
import numpy.random as random
test = dict()
train = dict()
acc = 0
wells = set(data['Well Name'])
similar_classes = dict()
for class_1 in range(9):
        for class_2 in range(class_1 + 1,9):
            similar_classes[class_1, class_2] = 0
sum_classes = dict()
for class_ in range(9):
        sum_classes[class_] = 0
for well in wells:
# well = 'SHRIMPLIN'
    print(well)
    test[well] = data[data['Well Name'] == well]
    train[well] = data[data['Well Name'] != well]
    X_train = train[well][feature_names].values 
    y_train = train[well]['Facies'].values 
    X_test = test[well][feature_names].values 
    y_test = test[well]['Facies'].values 

    robust = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_train)
    X_train_robust = robust.transform(X_train)
    X_test_robust = robust.transform(X_test)

    scaler = StandardScaler().fit(X_train_robust)
    X_train_robust_norm = scaler.transform(X_train_robust)
    X_test_robust_norm = scaler.transform(X_test_robust)
    
    dtrain = xgb.DMatrix(X_train_robust_norm, label=y_train)
    dtest = xgb.DMatrix(X_test_robust_norm, label=y_test)
    watchlist = [(dtest, 'eval'), (dtrain, 'train')]
    #     model = xgb.train(params, dtrain, 100)
    #     bst = xgb.train(param, dtrain, num_round, watchlist, obj=my_logregobj)
    model = xgb.Booster(params, [dtrain])
    for _ in range(150):
        pred = model.predict(dtrain)
        g, h = my_softmax(pred, dtrain)
        model.boost(dtrain, g, h)
    # Evalute
    yhat = model.predict(dtest)
    yhat_labels = np.argmax(yhat, axis=1)
    #     ypred = bst.predict(dtest)
    print(len(y_test), " ", len(yhat_labels))
    acc += f1_score(y_test, yhat_labels, average='micro')
    print(f1_score(y_test, yhat_labels, average='micro'))
#     for index in range(len(y_test)):
#         print(y_test[index], yhat_labels[index])
    most_similar_classes(yhat_labels, y_test)
    find_sum_classes(yhat_labels, y_test)
#     print("BAND 2 ", bandwidth_2(y_test))
#     print("BAND 3 ", bandwidth_3(y_test))
#     print("Before 2 test ", before_2(y_test))
#     print("After 2 test ", after_2(y_test))
#     print("Before 2 res ", before_2(yhat_labels))
#     print("After 2 res ", after_2(yhat_labels))
    
#     print("Before 3 test ", before_3(y_test))
#     print("After 3 test ", after_3(y_test))
#     print("Before 3 res ", before_3(yhat_labels))
#     print("After 3 res ", after_3(yhat_labels))
    print("Outliers test", number_of_outlier(y_test))
    print("Outliers res", number_of_outlier(yhat_labels))
print('well, boosting of trees, ', acc/10)  

SHRIMPLIN
471   471
0.5668789808917197
Outliers test 0.0021231422505307855
Outliers res 0.10615711252653928
SHANKLE
449   449
0.5077951002227171
Outliers test 0.0066815144766146995
Outliers res 0.05790645879732739
KIMZEY A
439   439
0.530751708428246
Outliers test 0.03189066059225513
Outliers res 0.10022779043280182
CROSS H CATTLE
501   501
0.3972055888223553
Outliers test 0.043912175648702596
Outliers res 0.0499001996007984
NOLAN
415   415
0.4963855421686747
Outliers test 0.05301204819277108
Outliers res 0.08433734939759036
LUKE G U
461   461
0.5835140997830802
Outliers test 0.006507592190889371
Outliers res 0.09327548806941431
NEWBY
463   463
0.4838012958963283
Outliers test 0.032397408207343416
Outliers res 0.0755939524838013
CHURCHMAN BIBLE
404   404
0.5420792079207921
Outliers test 0.05693069306930693
Outliers res 0.10643564356435643
Recruit F9
80   80
0.6625
Outliers test 0.0
Outliers res 0.1125
ALEXANDER D
466   466
0.6223175965665236
Outliers test 0.034334763948497854
Outliers 

In [20]:
recount_most_similar_classes()

In [21]:
similar_classes

{(0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0057553956834532375,
 (0, 5): 0.02631578947368421,
 (0, 6): 0.057233704292527825,
 (0, 7): 0.0482897384305835,
 (0, 8): 0.1752738654147105,
 (1, 2): 0.2818181818181818,
 (1, 3): 0.06069160197600565,
 (1, 4): 0.0,
 (1, 5): 0.0,
 (1, 6): 0.0015128593040847202,
 (1, 7): 0.0,
 (1, 8): 0.0,
 (2, 3): 0.41482078131292793,
 (2, 4): 0.001095890410958904,
 (2, 5): 0.007717750826901874,
 (2, 6): 0.0016750418760469012,
 (2, 7): 0.0,
 (2, 8): 0.0008305647840531562,
 (3, 4): 0.010796221322537112,
 (3, 5): 0.012236573759347382,
 (3, 6): 0.00293398533007335,
 (3, 7): 0.001557632398753894,
 (3, 8): 0.02711864406779661,
 (4, 5): 0.13776137761377613,
 (4, 6): 0.1802451333813987,
 (4, 7): 0.06389776357827476,
 (4, 8): 0.052594171997157074,
 (5, 6): 0.29941860465116277,
 (5, 7): 0.03902439024390244,
 (5, 8): 0.08882521489971347,
 (6, 7): 0.0656013456686291,
 (6, 8): 0.29035532994923857,
 (7, 8): 0.13234077750206782}